In [1]:
import pandas as pd
import requests
import json
import datetime

In [2]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

In [3]:
# Départ Montparnasse
#req = requests.get('https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:SNCF:87391003/departures',auth=(token, ''))

In [4]:
# Départ Montparnasse
req = requests.get('https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:SNCF:87391003/departures?count=50&',auth=(token, ''))

In [5]:
doc = json.loads(req.text)
row = len(doc['departures'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 50


In [6]:
df = pd.DataFrame(doc['departures'])
df_gare = pd.DataFrame(list(df['display_informations']))
df_heure = pd.DataFrame(list(df['stop_date_time']))

In [7]:
df_id = pd.DataFrame(list(df['links']))
df_id = pd.DataFrame(list(df_id[1]))
#df_id.head(5)

In [8]:
def del_day(str):
    return str[9:]

df_heure['departure_date_time'] = df_heure['departure_date_time'].apply(del_day)
df_heure['base_departure_date_time'] = df_heure['base_departure_date_time'].apply(del_day)
df_heure['retard'] = df_heure['departure_date_time'].astype('int32') - df_heure['base_departure_date_time'].astype('int32')
#df_heure.head(5)

In [9]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_heure['heure'] = df_heure['departure_date_time'].apply(str_tps)

In [10]:
def del_par(string):
    index = string.find("(")
    string = string[index+1:]
    return string[:-1]

df_gare['direction'] = df_gare['direction'].apply(del_par)

In [11]:
df_gare = df_gare[['direction','network','trip_short_name']]
df_gare['heure_depart'] = df_heure['heure']
df_gare['retard_min'] = df_heure['retard']
df_gare['id'] = df_id['id']

In [12]:
#df_gare.head(5)

In [13]:
def get_name(string):
    index_deb = string.find("'name': '")
    string = string[index_deb+9:]
    index_fin = string.find("', '")
    return string[:index_fin]

arrets = []

for i in df_gare.index:
    id = df_gare.iloc[i]['id']
    link = 'https://api.sncf.com/v1/coverage/sncf/vehicle_journeys/' + id
    req_arret = requests.get(link ,auth=(token, ''))
    doc = json.loads(req_arret.text)

    df_arret = pd.DataFrame(doc['vehicle_journeys'])
    df_arret = pd.DataFrame(list(df_arret['stop_times']))
    df_arret = df_arret.T
    df_arret[0] = df_arret[0].astype('str').apply(get_name)

    arrets.append(list(df_arret[0])[1:])

In [14]:
df_gare['arrets'] = arrets
df_gare = df_gare.drop(['id'], axis=1)

In [15]:
df_gare

,direction,network,trip_short_name,heure_depart,retard_min,arrets
0,Plaisir,TRANSILIEN,164501,00:05:00,0,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
1,Rambouillet,TRANSILIEN,164405,00:35:00,0,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
2,Versailles,TRANSILIEN,9002983,01:15:00,0,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
3,Rambouillet,TRANSILIEN,164443,05:20:00,0,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
4,Chartres,TER,862401,05:33:00,0,"[Versailles Chantiers, Rambouillet, Épernon, M..."
5,Mantes-la-Jolie,TRANSILIEN,164645,05:35:00,0,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
6,Bordeaux,TGV INOUI,8431,05:50:00,0,"[Saint-Pierre-des-Corps, Poitiers, Angoulême, ..."
7,Rambouillet,TRANSILIEN,164447,05:50:00,0,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
8,Plaisir,TRANSILIEN,164549,06:05:00,0,"[Vanves - Malakoff, Clamart, Meudon, Bellevue,..."
9,Toulouse,TGV INOUI,8501,06:06:00,0,"[Bordeaux Saint-Jean, Agen, Toulouse Matabiau]"
